# Capstone Project:The Battle of Neighborhoods (Week 2)

## Table of contents
* [Background](#Background)
* [Problem](#Probulem) 
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1) Background<a name="Background"></a>

I want to go back to my home town Toyota-city.  
However my wife was born in another area in Aichi, she is hesitating to go to Toyota-city.  
To overcome the situation I am planing to satisfy her dream by go back to Toyota-city.  
My wife's dream is to open a resturant.  
So showing the potential of Toyota-city for new resturant in this project.

## 2) Problem<a name="Problem"></a>

The idea of this study is to show which part of Toyota-city is the best for new resturant by providing data about:  
potential customers from population derived from population distribution. 
competitors in the same regions.

## 3) Data<a name="Data"></a>

For the study, population and resturant data in accordance with geographic information are needed.  
We can download population vs area name of Toyota-city from the URL below.

https://www.city.toyota.aichi.jp/_res/projects/default_project/_page_/001/004/759/h29/01.xls

Also we can download postcode list from the URL below.

https://www.post.japanpost.jp/zipcode/dl/roman-zip.html

Above two data can be linked by using area name and can be used to obtain geographic information(latitude & longitude) vs population.
In paticular, I used below API to get geographic information from postcode.

'http://geoapi.heartrails.com/api/xml?method=searchByPostal&postal='

Also we can get the resturant information by Foursquare API.
This resulting in linked data among geographic information, population and competiters for new resturant. 


### Post code and area name of Toyota-city

In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.error
import xml.etree.ElementTree
import requests

In [2]:
df = pd.read_csv("KEN_ALL_ROME.CSV", encoding='cp932')
df.head(10)

,Postcode,PrefJP,BoroughJP,AreaJP,Pref,Borough,Area
0,600000,北海道,札幌市 中央区,以下に掲載がない場合,HOKKAIDO,SAPPORO SHI CHUO KU,IKANIKEISAIGANAIBAAI
1,640941,北海道,札幌市 中央区,旭ケ丘,HOKKAIDO,SAPPORO SHI CHUO KU,ASAHIGAOKA
2,600041,北海道,札幌市 中央区,大通東,HOKKAIDO,SAPPORO SHI CHUO KU,ODORIHIGASHI
3,600042,北海道,札幌市 中央区,大通西（１～１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,ODORINISHI(1-19-CHOME)
4,640820,北海道,札幌市 中央区,大通西（２０～２８丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,ODORINISHI(20-28-CHOME)
5,600031,北海道,札幌市 中央区,北一条東,HOKKAIDO,SAPPORO SHI CHUO KU,KITA1-JOHIGASHI
6,600001,北海道,札幌市 中央区,北一条西（１～１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,KITA1-JONISHI(1-19-CHOME)
7,640821,北海道,札幌市 中央区,北一条西（２０～２８丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,KITA1-JONISHI(20-28-CHOME)
8,600032,北海道,札幌市 中央区,北二条東,HOKKAIDO,SAPPORO SHI CHUO KU,KITA2-JOHIGASHI
9,600002,北海道,札幌市 中央区,北二条西（１～１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,KITA2-JONISHI(1-19-CHOME)


In [3]:
df = df[df['Borough'] == 'TOYOTA SHI'].reset_index()
df = df[['Postcode', 'AreaJP', 'Pref', 'Borough', 'Area']]
df.AreaJP.astype(str)
df.head()

,Postcode,AreaJP,Pref,Borough,Area
0,4710000,以下に掲載がない場合,AICHI KEN,TOYOTA SHI,IKANIKEISAIGANAIBAAI
1,4710052,逢妻町,AICHI KEN,TOYOTA SHI,AIZUMACHO
2,4700335,青木町,AICHI KEN,TOYOTA SHI,AOKICHO
3,4710873,秋葉町,AICHI KEN,TOYOTA SHI,AKIBACHO
4,4710835,曙町,AICHI KEN,TOYOTA SHI,AKEBONOCHO


### Area name and population of Toyota-city

In [4]:
df_pop = pd.read_csv("01.csv", encoding='cp932')
#df_pop.AreaJP.astype(str)
df_pop.head()

,AreaJP,Population
0,逢妻町,684
1,秋葉町,"1,906"
2,曙町,"1,055"
3,朝日ケ丘,"2,560"
4,朝日町,"3,759"


In [5]:
df = pd.merge(df, df_pop,on='AreaJP', how='inner')
df.head()

,Postcode,AreaJP,Pref,Borough,Area,Population
0,4710052,逢妻町,AICHI KEN,TOYOTA SHI,AIZUMACHO,684
1,4700335,青木町,AICHI KEN,TOYOTA SHI,AOKICHO,"4,527"
2,4710873,秋葉町,AICHI KEN,TOYOTA SHI,AKIBACHO,"1,906"
3,4710835,曙町,AICHI KEN,TOYOTA SHI,AKEBONOCHO,"1,055"
4,4442832,浅谷町,AICHI KEN,TOYOTA SHI,AZAKAICHO,157


### Geographical data download

In [6]:
lat_long = []

def geo(zip_code, API_URL):
    zip = str(zip_code)
    response = urllib.request.urlopen(API_URL + zip)
    xml_result = response.read()
    response.close()
    elem = xml.etree.ElementTree.fromstring(xml_result)
    long = elem.findtext(".//x")
    lat = elem.findtext(".//y")
    data = [zip_code, lat, long]
    return data

URL = 'http://geoapi.heartrails.com/api/xml?method=searchByPostal&postal='

for i in range(len(df)):
    lat_long.append(geo(df.Postcode[i], URL))

df_lat_long = pd.DataFrame(lat_long, columns=['Postcode', 'Latitude', 'Longitude'])  
df_lat_long.head()

,Postcode,Latitude,Longitude
0,4710052,35.104364,137.139678
1,4700335,35.123342,137.185037
2,4710873,35.063098,137.165974
3,4710835,35.05884,137.133589
4,4442832,35.244875,137.372781


In [7]:
df = pd.merge(df, df_lat_long, on='Postcode')
df.head()

,Postcode,AreaJP,Pref,Borough,Area,Population,Latitude,Longitude
0,4710052,逢妻町,AICHI KEN,TOYOTA SHI,AIZUMACHO,684,35.104364,137.139678
1,4700335,青木町,AICHI KEN,TOYOTA SHI,AOKICHO,"4,527",35.123342,137.185037
2,4710873,秋葉町,AICHI KEN,TOYOTA SHI,AKIBACHO,"1,906",35.063098,137.165974
3,4710835,曙町,AICHI KEN,TOYOTA SHI,AKEBONOCHO,"1,055",35.05884,137.133589
4,4442832,浅谷町,AICHI KEN,TOYOTA SHI,AZAKAICHO,157,35.244875,137.372781


In [56]:
df = df[['Postcode', 'Area', 'Population', 'Latitude', 'Longitude']]
df.head(50)

,Postcode,Area,Population,Latitude,Longitude
0,4710052,AIZUMACHO,684,35.104364,137.139678
1,4700335,AOKICHO,"4,527",35.123342,137.185037
2,4710873,AKIBACHO,"1,906",35.063098,137.165974
3,4710835,AKEBONOCHO,"1,055",35.05884,137.133589
4,4442832,AZAKAICHO,157,35.244875,137.372781
5,4710858,ASAHIGAOKA,"2,560",35.079668,137.142553
6,4710031,ASAHIMACHI,"3,759",35.092834,137.140076
7,4442843,ASAHIYAWATACHO,35,35.209069,137.408441
8,4442423,AJIKYOCHO,13,35.116959,137.319574
9,4442817,ASUGACHO,37,35.213093,137.374024


### Foursquare API

In [18]:
#Input ID, Secret, version, limit, query and radius
CLIENT_ID = '1IXD'# your Foursquare ID
CLIENT_SECRET = 'SP5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
query_list = ['飲食店', 'レストラン', 'うどん', 'ラーメン', '料理', '定食', 'パスタ', 'カフェ', 'フードコート', '寿司', 'ステーキ', 'フード']
radius = 50000
res_info = []

In [19]:
latitude = df.Latitude[2]
longitude = df.Longitude[2]

for query in query_list:
    
#set URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, query, radius, LIMIT)
    
#get results
    results = requests.get(url).json()["response"]['venues']
#print(len(results))
    for i in range(len(results)):
        name = results[i]['name']
        category = results[i]['categories'][0]['name']
        lat = results[i]['location']['lat']
        lng = results[i]['location']['lng']
        info = [name, category, lat, lng]
        res_info.append(info)
        print("{},{},lat:{}lng:{}".format(name, category, lat, lng))

KeyError: 'venues'

## 4) Methodology<a name="Methodology"></a>

In this project I will search areas of Toyota city with low restaurant density and higher population density.  
As a first step I will get the restaurant information including location and category the post codes of each region in Toyota.  
Then plot the restaurant location in the map to visualize the current situation.  
Next I will calculate ratio of population vs restaurant number for each post codes.  
Finally I will summarize better places for new restaurant from the visual data.

### Restaurant data correction

In [60]:
#Due to the limitation of API I determined geographic center.
latitude = df.Latitude[12]
longitude = df.Longitude[12]
    
for query in query_list:
    
#set URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, query, radius, LIMIT)
    
#get results
    results = requests.get(url).json()["response"]['venues']
        
        
#print(len(results))
    for i in range(len(results)):
        name = results[i]['name']
        category = results[i]['categories'][0]['name']
        lat = results[i]['location']['lat']
        lng = results[i]['location']['lng']
        info = [name, category, lat, lng]
        res_info.append(info)
    #print("{},{},lat:{}lng:{}".format(name, category, lat, lng))

KeyError: 'venues'

In [21]:
dname = ["name", "category", "Latitude", "Longitude"]
df_res = pd.DataFrame(res_info, columns=dname)
print(len(res_info))
df_res.head()

0


,name,category,Latitude,Longitude


### Data visualization

In [22]:
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [51]:
address  =  'Toyota'
geolocator = Nominatim(user_agent="Toyota_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toyota_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, name, category in zip(df_res['Latitude'], df_res['Longitude'], df_res['name'], df_res['category']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toyota_map)
    
Toyota_map

### Analysis of visual data

As shown in the map almost all the restaurant is in urban area of the Toyota city.  
My assumption on the reason of this distribution is among mountains(green colored) lower population density  
resulting in the difficulty in attracting customers to restaurants.  
To confirm this assumption, I plot the population of each area in the bubble charts.

In [61]:
import math

address  =  'Toyota'
geolocator = Nominatim(user_agent="Toyota_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toyota_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, population, postcode, area in zip(df['Latitude'], df['Longitude'], df['Population'], df['Postcode'], df['Area']):
    population=math.ceil(float(population.replace(",", ""))/150)
    if population == 0:
        break
    else:
        #print(population)
        label = '{}, {}'.format(postcode, area)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=population,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(Toyota_map)
    
Toyota_map

As expected, most of people living in central Toyota-city.  
However there are three big circles can be seen in moutain are.  
These area has relatively higher population density and no restaurant.  
So I will focus on these 3 areas.
To judge which one is the best place for new restaurant, I should set the criteria.  
For this I set the ratio of population vs restaurant in urban area of Toyota-city as the criteria value.  
Next I calculate the ratio of population vs restaurant for these 3 areas plus urban area.